### Homework #1 For MLOPS Zoomcamp
Student: Carlos Velázquez

In [1]:
#Installing necessary libraries
!pip install -U scikit-learn


In [2]:
# Importing necessary libraries
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_parquet('/home/carlosv/mlopszoomcamp/yellow_tripdata_2022-01.parquet')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463926,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,None,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,NaN,NaN
2463927,2,2022-01-31 23:44:22,2022-01-31 23:55:01,NaN,4.19,NaN,None,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,NaN,NaN
2463928,2,2022-01-31 23:39:00,2022-01-31 23:50:00,NaN,2.10,NaN,None,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,NaN,NaN
2463929,2,2022-01-31 23:36:42,2022-01-31 23:48:45,NaN,2.92,NaN,None,148,164,0,12.40,0.0,0.5,0.00,0.0,0.3,15.70,NaN,NaN


#### Question 1: Number of columns in Jan 2022 Yellow Taxi Trip data


In [4]:
len(df.columns)

19

##### Question 2:  Standard deviation of the trips duration in Jan 2022 Yellow Taxi Trip data

In [5]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
(df.duration.dt.total_seconds() / 60 ).std()


46.44530513776802

#### Question 3: Fraction of the records left after dropping the outliers

In [6]:
# First we'll get the number of records before dropping the outliers
total = df.shape[0]


In [7]:
# We'll transform the duration column to minutes because the possible answers are in minuted
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)


In [8]:
#Removing outliers
df = df[(df.duration >= 1) & (df.duration <= 60)]


In [9]:
#Getting the new number of records
total_new = df.shape[0]

In [10]:
# Calculating answer 
(total_new * 100 )/ total

98.27547930522405

#### Question 4: Dimensionality after OHE

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [11]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)
#Turning Dataframe into list of dictionaries
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

/tmp/ipykernel_16280/1443212928.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [12]:
X_train.shape[1]

515

#### Question 5: RMSE on train

In [13]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190837370544

Question 6: RMSE on validation

In [14]:
#Reading Validation data
df_valid = pd.read_parquet('/home/carlosv/mlopszoomcamp/yellow_tripdata_2022-02.parquet')
df_valid

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.00,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.00,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.50,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.00,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.50,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2979426,2,2022-02-28 23:50:00,2022-03-01 00:06:00,NaN,3.40,NaN,None,163,193,0,14.37,0.00,0.5,0.00,0.00,0.3,17.67,NaN,NaN
2979427,2,2022-02-28 23:06:57,2022-02-28 23:19:12,NaN,3.48,NaN,None,141,4,0,14.51,0.00,0.5,2.00,0.00,0.3,19.81,NaN,NaN
2979428,2,2022-02-28 23:48:13,2022-03-01 00:03:33,NaN,3.05,NaN,None,161,151,0,14.38,0.00,0.5,3.81,0.00,0.3,21.49,NaN,NaN
2979429,2,2022-02-28 23:56:41,2022-03-01 00:04:57,NaN,2.62,NaN,None,141,226,0,12.53,0.00,0.5,1.71,0.00,0.3,17.54,NaN,NaN


In [15]:
df_valid['duration'] = df_valid.tpep_dropoff_datetime - df_valid.tpep_pickup_datetime
df_valid.duration = df_valid.duration = df_valid.duration.apply(lambda td: td.total_seconds() / 60)

df_valid = df_valid[(df_valid.duration >= 1) & (df_valid.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df_valid[categorical] = df_valid[categorical].astype(str)

/tmp/ipykernel_16280/2296352548.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid[categorical] = df_valid[categorical].astype(str)


In [16]:
#Removing outliers
df_valid = df_valid[(df_valid.duration >= 1) & (df_valid.duration <= 60)]


In [17]:
categorical = ['PULocationID', 'DOLocationID']
df_valid[categorical] = df_valid[categorical].astype(str)
#Turning Dataframe into list of dictionaries
valid_dicts = df_valid[categorical].to_dict(orient='records')

In [18]:
X_valid = dv.transform(valid_dicts)

In [19]:
target = 'duration'
y_valid = df_valid[target].values



y_pred = lr.predict(X_valid)

mean_squared_error(y_valid, y_pred, squared=False)

7.786409085078911